In [42]:
############ LOADING ###############

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import csv
import pycatch22

# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=RuntimeWarning)
simplefilter(action='ignore', category=FutureWarning)

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

%matplotlib inline

# variavel que evita NaN nos resultados
epslon = 0.00005

def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_pred - y_true)/np.sum(y_true))
  else:
   return 100*(np.sum(y_pred - y_true)/(np.sum(y_true)+ epslon))  

def pocid(y_true, y_pred):
    n = len(y_true)
    D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
    POCID = 100 * np.sum(D) / n
    return POCID

#função para normalização
def znorm(x):
  if np.std(x)!=0:
    x_znorm = (x - np.mean(x)) / np.std(x)
  else:
    x_znorm = (x - np.mean(x)) / (np.std(x) + epslon) 
   
  return x_znorm

#função para desnormatização
def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Em geral, considera-se um tamanho de janela capaz de capturar um ciclo dos dados
# Por exemplo, 12 observações no caso dos dados com frequência mensal
def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds


In [43]:
#VERIFICAÇÃO DOS ESTADOS 

def extract_estado(file_name):
    # Split the file name by underscores
    parts = file_name.split('_')
    # Extract the name between underscores
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    # List all files in the folder
    files = os.listdir(folder_path)
    # Iterate through each file
    for file_name in files:
        # Check if it's a CSV file
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Open the CSV file and read the data
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                # Assuming the first row contains headers
                headers = next(reader)
                # Extract estado from file name and append to estados list
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados



In [49]:
############## AUTO ##############

horizon = 12
window = 12

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    for estado in estados:
        #####################################
        products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
        folder_path = f'./uf/{product}/'
        # Read the CSV files and extract estado names
        estados = read_csv_files(folder_path)
        # Carregamento do arquivo csv com todas as features
        df_c = pd.read_csv(f'./concatenate_12/{product}_{window}_{estado}_concatenated.csv')
        df_c_ts = df_c.iloc[:, 0]
        df_features = df_c.drop(df_c.columns[0], axis=1)
        df_t = pd.read_csv(f'./uf/{product}/mensal_{estado}_{product}.csv', header=0, sep=";")
        series = df_t['m3']
        target = rolling_window(series, window)

        # Exclude the first 24 rows and get the 'm3' column
        target_norm = target.iloc[:, -1].tail(386).reset_index(drop=True)
        df_c_znorm = df_features

        df_c_znorm['y'] = target_norm

        #### Possível seleção de Conjunto de Features em df_c_znorm (wind=12) ####
        # timestamp = 1
        # catch22 = 22 
        # cesium = 117 
        # tsfeatures = 42 
        # tsfel = 140
        # tsflex = 14 
        # tsfresh = 783

        ##################### CORRELATION ########################################################
        columns_ext = df_c_znorm.columns[:-1]
        dataE = pd.DataFrame({f'{i+1}': df_c_znorm[col] for i, col in enumerate(columns_ext)})
        dataE['y'] = df_t['m3'].tail(386)
        pearson_corr = dataE.corr(method='pearson')
        spearman_corr = dataE.corr(method='spearman')
        kendall_corr = dataE.corr(method='kendall')
        correlation_with_y = pd.DataFrame({
            'Pearson': pearson_corr['y'].drop('y'),
            'Spearman': spearman_corr['y'].drop('y'),
            'Kendall': kendall_corr['y'].drop('y')
        })
        correlation_with_y['Pearson2'] = correlation_with_y['Pearson'].abs()
        correlation_with_y['Spearman2'] = correlation_with_y['Spearman'].abs()
        correlation_with_y['Kendall2'] = correlation_with_y['Kendall'].abs()
        correlation_with_y['total'] = correlation_with_y['Pearson2'] + correlation_with_y['Spearman2'] + correlation_with_y['Kendall2']
        DataTBI = correlation_with_y['total'][(correlation_with_y['Pearson2'] > 0.3) | (correlation_with_y['Spearman2'] > 0.3) | (correlation_with_y['Kendall2'] > 0.3)]
        DataTBI_sorted = DataTBI.sort_values(ascending=False)
        DataTBI_SelCol = dataE.loc[:, DataTBI_sorted.index]
                
        DataTBI_SelCol = DataTBI_SelCol.apply(znorm, axis=1)
        DataTBI_SelCol.replace([np.inf, -np.inf], 0, inplace=True)
        DataTBI_SelCol = DataTBI_SelCol.fillna(0)
        DataTBI_SelCol['y'] = target_norm

        ##################################################################################
        # Divisão dos dados em treino/teste considerando o horizonte de predição de 12 meses
        X_train, X_test, y_train, y_test = train_test_split(DataTBI_SelCol, horizon)

        #######################  lgb  #######################
        num_round = 100
        # Convert data to LightGBM dataset format
        train_data = lgb.Dataset(X_train, label=y_train)
        test_data = lgb.Dataset(X_test, label=y_test)

        ############## Regressores ##################
        # regr1 = LinearRegression()
        regr2 = KNeighborsRegressor(n_neighbors = 3)
        regr3 = XGBRegressor()
        regr4 = SVR(kernel='rbf')
        regr5 = RandomForestRegressor()
        # regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)
        regr7 = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', metric='rmse', 
                                num_leaves=31, learning_rate=0.05, feature_fraction=0.9, 
                                bagging_fraction=0.8, bagging_freq=5, verbose=-1)
        # Treinamento dos modelos
        # regr1.fit(X_train, y_train)
        # predictions1a = recursive_multistep_forecasting(X_test, regr1, horizon)
        regr2.fit(X_train, y_train)
        predictions2a = recursive_multistep_forecasting(X_test, regr2, horizon)
        regr3.fit(X_train, y_train)
        predictions3a = recursive_multistep_forecasting(X_test, regr3, horizon)
        regr4.fit(X_train, y_train)
        predictions4a = recursive_multistep_forecasting(X_test, regr4, horizon)
        regr5.fit(X_train, y_train)
        predictions5a = recursive_multistep_forecasting(X_test, regr5, horizon)
        # regr6.fit(X_train, y_train)
        # predictions6a = recursive_multistep_forecasting(X_test, regr6, horizon)
        regr7.fit(X_train, y_train)
        predictions7a = recursive_multistep_forecasting(X_test, regr7, horizon)
        # Recupera a média e desvio-padrão da última subsequência observada
        mean_norm, std_norm = get_stats_norm(series, horizon, window)
        # Reescala a predição
        # predictions1 = znorm_reverse(predictions1a, mean_norm, std_norm)
        predictions2 = znorm_reverse(predictions2a, mean_norm, std_norm)
        predictions3 = znorm_reverse(predictions3a, mean_norm, std_norm)
        predictions4 = znorm_reverse(predictions4a, mean_norm, std_norm)
        predictions5 = znorm_reverse(predictions5a, mean_norm, std_norm)
        # predictions6 = znorm_reverse(predictions6a, mean_norm, std_norm)
        predictions7 = znorm_reverse(predictions7a, mean_norm, std_norm)
        ##########################
        valores_reais = series.tail(12)
        valores_reais.reset_index(inplace=True, drop=True)
        # mape_result1 = mape(valores_reais, predictions1)
        mape_result2 = mape(valores_reais, predictions2)
        mape_result3 = mape(valores_reais, predictions3)
        mape_result4 = mape(valores_reais, predictions4)
        mape_result5 = mape(valores_reais, predictions5)
        # mape_result6 = mape(valores_reais, predictions6)
        mape_result7 = mape(valores_reais, predictions7)
        # pbe_result1 = pbe(valores_reais, predictions1)
        pbe_result2 = pbe(valores_reais, predictions2)
        pbe_result3 = pbe(valores_reais, predictions3)
        pbe_result4 = pbe(valores_reais, predictions4)
        pbe_result5 = pbe(valores_reais, predictions5)
        # pbe_result6 = pbe(valores_reais, predictions6)
        pbe_result7 = pbe(valores_reais, predictions7)
        # pocid_result1 = pocid(valores_reais, predictions1)
        pocid_result2 = pocid(valores_reais, predictions2)
        pocid_result3 = pocid(valores_reais, predictions3)
        pocid_result4 = pocid(valores_reais, predictions4)
        pocid_result5 = pocid(valores_reais, predictions5)
        # pocid_result6 = pocid(valores_reais, predictions6)
        pocid_result7 = pocid(valores_reais, predictions7)
        # p1 = ', '.join(map(str, predictions1))
        p2 = ', '.join(map(str, predictions2))
        p3 = ', '.join(map(str, predictions3))
        p4 = ', '.join(map(str, predictions4))
        p5 = ', '.join(map(str, predictions5))
        # p6 = ', '.join(map(str, predictions6))
        p7 = ', '.join(map(str, predictions7))
        rows_data = [
            # [product,estado,'LR',fd1,mape_result1,pocid_result1,pbe_result1,p1],
            [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,p2],
            [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,p3],
            [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,p4],
            [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,p5],
            # [product,estado,'MLP',fd6,mape_result6,pocid_result6,pbe_result6,p6],
            [product,estado,'LGB',mape_result7,pocid_result7,pbe_result7,p7]             
        ]
        # CSV Output VALORES REAIS
        with open(f'Metrics_ALL_{product}_{window}_output.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            for row_data in rows_data:
                writer.writerow(row_data)

KeyError: 'Average'

In [48]:
dataE

,1,2,3,4,5,6,7,8,9,10,...,1110,1111,1112,1113,1114,1115,1116,1117,1118,y
0,0.523417,0.259304,-0.903344,-1.213283,-0.377968,-1.048322,-1.185829,0.340350,-1.431287,-0.310791,...,1.240373,0.663710,-2.062046,-1.191792,0.442998,0.248364,0.156307,0.0,1.784139,-0.097247
1,0.535407,0.270821,-0.905549,-1.213283,-0.465339,-1.045800,-1.185829,0.340350,-1.431287,-0.310791,...,1.240373,0.663710,-0.905042,-0.517680,0.442998,0.248364,0.156307,0.0,1.767614,-0.020284
2,0.488095,0.223789,-0.919377,-1.213283,-0.120885,-1.064798,0.616258,0.340350,-1.135854,-0.310791,...,0.627686,0.663710,-1.079825,-0.517680,0.442998,0.248364,0.156307,0.0,1.780129,-0.109061
3,0.452404,0.187956,-0.914307,-1.213283,-0.962053,-1.095184,0.616258,0.340350,-1.283570,-0.310791,...,0.627686,0.663710,-0.810680,-0.263240,0.442998,0.248364,0.156307,0.0,1.780129,0.243171
4,0.435297,0.172291,-0.898342,-1.213283,-0.962053,-0.612622,0.616258,0.340350,-1.135854,-0.310791,...,1.240373,0.663710,0.346325,-0.263240,0.442998,0.248364,0.156307,0.0,1.811882,-0.201873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,-0.844894,-1.001394,0.755049,-0.267167,-0.465339,0.132790,0.616258,-1.427029,1.301469,-0.310791,...,-1.210376,0.663710,1.328545,1.084985,0.442998,0.248364,0.156307,0.0,0.184777,2.143991
382,-0.384026,-1.071411,1.392493,2.098121,0.136198,0.002436,0.616258,-0.837902,1.301469,-0.310791,...,-1.060198,0.663710,0.346325,0.410872,0.442998,0.248364,0.156307,0.0,0.528702,1.491531
383,0.082933,-1.183579,1.928462,2.571179,0.885179,0.005892,0.616258,-0.248776,1.301469,-0.310791,...,-1.060198,-0.601958,0.077179,0.410872,0.442998,0.248364,0.156307,0.0,0.786221,1.605760
384,2.098482,0.259180,2.269848,2.098121,0.658132,-0.005834,0.616258,-0.248776,1.301469,-0.310791,...,-1.703479,-3.611014,-0.905042,-0.263240,0.442998,0.248364,0.156307,0.0,1.150710,1.275466


In [41]:
############## AUTO ##############

horizon = 12
window = 12


products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    
    for estado in estados:


        ########################################
        products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])

        folder_path = f'./uf/{product}/'
        # Read the CSV files and extract estado names
        estados = read_csv_files(folder_path)

        # Carregamento do arquivo csv com todas as features
        df_c = pd.read_csv(f'./concatenate/{product}_{window}_{estado}_concatenated.csv')

        df_c_ts = df_c.iloc[:, 0]
        df_features = df_c.drop(df_c.columns[0], axis=1)

        df_t = pd.read_csv(f'./uf/{product}/mensal_{estado}_{product}.csv', header=0, sep=";")

        series = df_t['m3']
        target = rolling_window(series, window)

        # Exclude the first 24 rows and get the 'm3' column
        target_norm = target.iloc[:, -1].tail(386).reset_index(drop=True)

        # Normalization of features
        df_c_znorm = df_features.apply(znorm, axis=0)
        df_c_znorm.fillna(0, inplace=True)
        df_c_znorm['y'] = target_norm

        #### Possível seleção de Conjunto de Features em df_c_znorm (wind=12) ####

        # timestamp = 1
        # catch22 = 22 
        # cesium = 117 
        # tsfeatures = 42 
        # tsfel = 140
        # tsflex = 14 
        # tsfresh = 783

        ##################### CORRELATION ########################################################



        #### XXXX



        ##################################################################################
        # Divisão dos dados em treino/teste considerando o horizonte de predição de 12 meses
        X_train, X_test, y_train, y_test = train_test_split(df_c_znorm, horizon)

        #######################  lgb  #######################
        num_round = 100

        # Convert data to LightGBM dataset format
        train_data = lgb.Dataset(X_train, label=y_train)
        test_data = lgb.Dataset(X_test, label=y_test)

        ############## Regressores ##################
        # regr1 = LinearRegression()
        regr2 = KNeighborsRegressor(n_neighbors = 3)
        regr3 = XGBRegressor()
        regr4 = SVR(kernel='rbf')
        regr5 = RandomForestRegressor()
        # regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)
        regr7 = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', metric='rmse', 
                                num_leaves=31, learning_rate=0.05, feature_fraction=0.9, 
                                bagging_fraction=0.8, bagging_freq=5, verbose=-1)

        # Treinamento dos modelos
        # regr1.fit(X_train, y_train)
        # predictions1a = recursive_multistep_forecasting(X_test, regr1, horizon)
        regr2.fit(X_train, y_train)
        predictions2a = recursive_multistep_forecasting(X_test, regr2, horizon)
        regr3.fit(X_train, y_train)
        predictions3a = recursive_multistep_forecasting(X_test, regr3, horizon)
        regr4.fit(X_train, y_train)
        predictions4a = recursive_multistep_forecasting(X_test, regr4, horizon)
        regr5.fit(X_train, y_train)
        predictions5a = recursive_multistep_forecasting(X_test, regr5, horizon)
        # regr6.fit(X_train, y_train)
        # predictions6a = recursive_multistep_forecasting(X_test, regr6, horizon)
        regr7.fit(X_train, y_train)
        predictions7a = recursive_multistep_forecasting(X_test, regr7, horizon)

        # Recupera a média e desvio-padrão da última subsequência observada
        mean_norm, std_norm = get_stats_norm(series, horizon, window)

        # Reescala a predição
        # predictions1 = znorm_reverse(predictions1a, mean_norm, std_norm)
        predictions2 = znorm_reverse(predictions2a, mean_norm, std_norm)
        predictions3 = znorm_reverse(predictions3a, mean_norm, std_norm)
        predictions4 = znorm_reverse(predictions4a, mean_norm, std_norm)
        predictions5 = znorm_reverse(predictions5a, mean_norm, std_norm)
        # predictions6 = znorm_reverse(predictions6a, mean_norm, std_norm)
        predictions7 = znorm_reverse(predictions7a, mean_norm, std_norm)


        ##########################

        valores_reais = series.tail(12)
        valores_reais.reset_index(inplace=True, drop=True)

        # mape_result1 = mape(valores_reais, predictions1)
        mape_result2 = mape(valores_reais, predictions2)
        mape_result3 = mape(valores_reais, predictions3)
        mape_result4 = mape(valores_reais, predictions4)
        mape_result5 = mape(valores_reais, predictions5)
        # mape_result6 = mape(valores_reais, predictions6)
        mape_result7 = mape(valores_reais, predictions7)

        # pbe_result1 = pbe(valores_reais, predictions1)
        pbe_result2 = pbe(valores_reais, predictions2)
        pbe_result3 = pbe(valores_reais, predictions3)
        pbe_result4 = pbe(valores_reais, predictions4)
        pbe_result5 = pbe(valores_reais, predictions5)
        # pbe_result6 = pbe(valores_reais, predictions6)
        pbe_result7 = pbe(valores_reais, predictions7)

        # pocid_result1 = pocid(valores_reais, predictions1)
        pocid_result2 = pocid(valores_reais, predictions2)
        pocid_result3 = pocid(valores_reais, predictions3)
        pocid_result4 = pocid(valores_reais, predictions4)
        pocid_result5 = pocid(valores_reais, predictions5)
        # pocid_result6 = pocid(valores_reais, predictions6)
        pocid_result7 = pocid(valores_reais, predictions7)

        # p1 = ', '.join(map(str, predictions1))
        p2 = ', '.join(map(str, predictions2))
        p3 = ', '.join(map(str, predictions3))
        p4 = ', '.join(map(str, predictions4))
        p5 = ', '.join(map(str, predictions5))
        # p6 = ', '.join(map(str, predictions6))
        p7 = ', '.join(map(str, predictions7))

        
       
        rows_data = [
            # [product,estado,'LR',fd1,mape_result1,pocid_result1,pbe_result1,p1],
            [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,p2],
            [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,p3],
            [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,p4],
            [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,p5],
            # [product,estado,'MLP',fd6,mape_result6,pocid_result6,pbe_result6,p6],
            [product,estado,'LGB',mape_result7,pocid_result7,pbe_result7,p7]             
        ]
        
        # CSV Output VALORES REAIS
        with open(f'Metrics_ALL_{product}_{window}_output.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            for row_data in rows_data:
                writer.writerow(row_data)

NameError: name 'merged_df' is not defined

In [24]:
df_c_znorm

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.465632
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.418250
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.804121
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.256665
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.032866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.169306
382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.140152
383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.428244
384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.683579


In [23]:
columns_ext

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '773', '774', '775', '776', '777', '778', '779', '780', '781', '782'],
      dtype='object', length=1118)

In [28]:

df_c

,timestamp,0,1,2,3,4,5,6,7,8,...,773,774,775,776,777,778,779,780,781,782
0,199201,-0.207302,0.174793,0.606041,1,0.693147,-2.300518,1.000000,3.0,0.030303,...,1.277034,1.475076,1.94591,1.504788,2.043192,2.079442,1.94591,1.791759,0.0,9136.859000
1,199202,-0.105213,-0.303797,0.618322,1,0.693147,-2.417237,1.000000,3.0,0.022039,...,1.078992,1.549826,1.94591,1.504788,2.043192,2.079442,1.94591,1.791759,0.0,8857.642429
2,199203,-0.081495,-0.278593,0.621159,1,0.693147,-2.360276,1.000000,3.0,0.013774,...,1.277034,1.549826,1.94591,1.504788,2.043192,2.079442,1.94591,1.791759,0.0,8857.642429
3,199204,-0.063490,-0.258444,0.642899,1,0.693147,-2.310760,1.000000,3.0,0.111111,...,1.277034,1.747868,1.94591,1.557113,2.043192,2.079442,1.94591,1.791759,0.0,8857.642429
4,199205,-0.064570,-0.259481,0.642416,1,0.693147,-2.191727,1.000000,3.0,0.111111,...,1.549826,1.945910,1.94591,1.748067,2.043192,2.079442,1.94591,1.791759,0.0,8860.805286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,202310,0.425640,0.479703,0.698260,2,0.838699,0.778143,0.909091,4.0,0.080000,...,1.277034,1.549826,1.94591,1.643418,2.043192,2.079442,1.94591,1.791759,0.0,5580.162571
382,202311,0.938797,0.441500,0.662414,2,1.054920,0.710838,0.909091,4.0,0.120000,...,0.955700,1.351784,1.94591,1.695743,2.043192,2.079442,1.94591,1.791759,0.0,5593.184143
383,202312,1.037716,0.252853,0.686097,2,0.804719,0.567091,0.909091,4.0,0.040000,...,0.955700,1.277034,1.94591,1.695743,2.043192,2.079442,1.94591,1.791759,0.0,5744.069571
384,202401,0.225694,0.047631,0.870018,2,0.725336,0.657773,0.909091,4.0,0.040000,...,1.549826,1.549826,1.94591,1.504788,1.889159,2.079442,1.94591,1.791759,0.0,5934.338571


In [22]:
df_c_ts

0      199201
1      199202
2      199203
3      199204
4      199205
        ...  
381    202310
382    202311
383    202312
384    202401
385    202402
Name: timestamp, Length: 386, dtype: int64

In [ ]:
df_c_znorm

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,y
0,-0.246839,-0.589506,0.048393,0.549425,-1.281594,-0.820101,0.467134,-0.611513,-0.225369,-0.445823,...,1.033322,0.585361,-0.197712,1.061748,0.410004,0.153707,0.102329,0.0,-1.364229,2455.831
1,-0.339209,-0.019474,-0.129771,1.096018,-2.403133,-0.783326,0.467134,-0.611513,-0.225369,-0.445823,...,1.033322,-1.135965,0.089040,1.061748,0.410004,0.153707,0.102329,0.0,-1.362098,2110.210
2,-1.093684,-1.051946,-0.382254,-0.543761,-0.160056,-0.983032,0.467134,-0.611513,-0.755352,-0.445823,...,0.319119,-1.135965,1.135513,1.061748,0.410004,0.153707,0.102329,0.0,-1.362098,2245.507
3,-0.850453,-1.124251,-0.400006,0.549425,-0.716361,-1.094968,0.467134,-0.611513,-0.225369,2.243045,...,1.033322,0.585361,0.848761,1.061748,0.410004,0.153707,0.102329,0.0,-1.362098,2448.490
4,-0.964566,-0.626619,-0.320923,-0.543761,-0.590377,-1.246471,0.467134,-0.611513,1.214804,-0.445823,...,1.033322,0.585361,0.848761,1.061748,0.410004,0.153707,0.102329,0.0,-1.360171,2261.490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,-1.519490,-1.735806,-0.885453,-1.090354,0.531161,0.558451,0.467134,0.205248,-1.163266,-0.445823,...,1.033322,0.585361,0.848761,1.061748,0.410004,0.153707,0.102329,0.0,1.767920,35301.000
382,-1.527852,-1.743738,-0.897765,-1.090354,0.009370,0.614196,-1.581887,1.022010,-1.291810,-0.445823,...,0.319119,-1.135965,1.135513,1.061748,0.410004,0.153707,0.102329,0.0,1.769398,34727.055
383,-0.450490,-0.151739,-0.722950,-1.090354,-0.420951,-1.341866,-1.581887,1.022010,-1.291810,-0.445823,...,1.033322,0.585361,0.848761,0.152499,0.410004,0.153707,0.102329,0.0,1.769398,39848.000
384,0.728141,0.202450,-0.524836,-1.090354,0.270266,1.770406,-1.581887,1.022010,0.350700,-0.445823,...,1.033322,0.585361,-0.197712,0.152499,0.410004,0.153707,0.102329,0.0,1.764698,35655.300
